In [9]:
import os
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail, Email, To, Content, Subject
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, trace, Runner, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput, function_tool
from typing import Dict
from pydantic import BaseModel

In [4]:
load_dotenv(override=True)

True

In [5]:
openai_api_key = os.getenv('OPENAI_API_KEY')
gemini_api_key = os.getenv('GEMINI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=gemini_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

gemini_model = OpenAIChatCompletionsModel(model='gemini-2.5-flash', openai_client=gemini_client)
groq_model = OpenAIChatCompletionsModel(model='llama-3.3-70b-versatile', openai_client=groq_client)

In [6]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [7]:
sales_agent1 = Agent(name='Gemini sales agent', instructions=instructions1, model=gemini_model)
sales_agent2 = Agent(name='Llama-3.3 sales Agent', instructions=instructions2, model=groq_model)
sales_agent3 = Agent(name='OpenAI sales agent', instructions=instructions3, model='gpt-4o-mini')

In [10]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [11]:
@function_tool
def send_html_email(body: str, subject: str):
    """ Send out an email with the given body to all sales prospects """
    sg = SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("soroushav85@gmail.com")
    to = To('soroush.alahvaisi@studenti.unime.it')
    content = Content('text/html', content=body)
    mail = Mail(from_email=from_email, to_emails=to, subject=subject, plain_text_content=content)
    response = sg.send(message=mail)
    return {"status": "success"}

In [12]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email Subject Writer", instructions=subject_instructions, model="gpt-4o-mini")
html_converter = Agent(name="Html email body converter", instructions=html_instructions, model="gpt-4o-mini")

subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for an email")
html_converter_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Convert a text email body to a html body")

In [13]:
email_tools = [subject_tool, html_converter_tool, send_html_email]

In [14]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [15]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [16]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name='Sales Manager',
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model='gpt-4o-mini'
)

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

In [17]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent(
    name="Nmae check",
    instructions='Check whether user is including any personal name in what they want to do .',
    output_type=NameCheckOutput,
    model='gpt-4o-mini'
)


In [18]:
@input_guardrail
async def guardrail_against_names(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output}, tripwire_triggered=is_name_in_message)


In [19]:
careful_sales_manager = Agent(
    name='Sales Manager',
    instructions=sales_manager_instructions,
    tools=tools,
    model='gpt-4o-mini',
    handoffs=handoffs,
    input_guardrails=[guardrail_against_names]
)



In [ ]:

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message) # it will have an error

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

In [21]:

message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)